In [1]:
import numpy as np
import pandas as pd
import optuna

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
RAND_STATE = 42

In [3]:
# Mock dataset for demonstration
class TextDataset(Dataset):
    # def __init__(self, vocabulary_size, sequence_length, num_samples):
    #     self.data = torch.randint(0, vocabulary_size, (num_samples, sequence_length))
    #     self.labels = torch.randint(0, 2, (num_samples,))
    def __init__(self, texts, labels, sequence_length):
        self.texts = texts
        self.labels = labels
        self.sequence_length = sequence_length
        self.vocab = self.build_vocab(texts)
        self.encoded_texts = [self.encode_text(text) for text in texts]

    def build_vocab(self, texts):
        unique_words = set(word for text in texts for word in text.lower().split())
        vocab = {word: i + 1 for i, word in enumerate(unique_words)}  # +1 for padding token at index 0
        return vocab
    
    def encode_text(self, text):
        return [self.vocab.get(word, 0) for word in text.lower().split()][:self.sequence_length] + [0] * (self.sequence_length - len(text.split()))

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.encoded_texts[idx]), torch.tensor(self.labels[idx])
    

# Define the Q-network model
# class DQN(nn.Module):
#     def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes):
#         super(DQN, self).__init__()
#         self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, num_classes)

#     def forward(self, x):
#         embeds = self.embedding(x)
#         lstm_out, _ = self.lstm(embeds)
#         q_values = self.fc(lstm_out[:, -1])
#         return q_values

# class DQN(nn.Module):
#     def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes):
#         super(DQN, self).__init__()
#         self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, num_classes)

#     def forward(self, x):
#         embeds = self.embedding(x)
#         lstm_out, _ = self.lstm(embeds)
#         q_values = self.fc(lstm_out[:, -1])
#         return q_values
    
class PolicyNetwork(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes, dropout_rate=0.5, pre_trained_embeddings=None):
        super(PolicyNetwork, self).__init__()
        self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
        if pre_trained_embeddings is not None:
            self.embedding.weight = nn.Parameter(pre_trained_embeddings)
            self.embedding.weight.requires_grad = False  # Or True if you want to fine-tune

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, _ = self.lstm(embeds)
        out = self.dropout(lstm_out[:, -1])
        logits = self.fc(out)
        probabilities = self.softmax(logits)
        return probabilities

def policy_gradient_loss(probabilities, actions, rewards):
    # Negative log likelihood loss
    log_prob = torch.log(probabilities[range(len(actions)), actions])
    loss = -torch.mean(log_prob * rewards)
    return loss

In [4]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    for batch_idx, (data, actions) in enumerate(train_loader):
        data, actions = data.to(device), actions.to(device)
        optimizer.zero_grad()
        probabilities = model(data)
        
        # Simulate rewards; in a real RL scenario, this would come from the environment
        rewards = torch.rand(len(actions))  # Placeholder for actual reward calculation logic
        
        loss = policy_gradient_loss(probabilities, actions, rewards)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def validate(model, device, validation_loader):
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            validation_loss += nn.CrossEntropyLoss()(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    validation_loss /= len(validation_loader.dataset)
    validation_acc = correct / len(validation_loader.dataset)
    print(f'\nValidation set: Average loss: {validation_loss:.4f}, Accuracy: {correct}/{len(validation_loader.dataset)} ({100. * correct / len(validation_loader.dataset):.0f}%)\n')
    return validation_loss, validation_acc

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss()(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [5]:
df = pd.read_csv('dataset/sentiment_analysis.csv')

# Extracting texts and labels
texts = df['tweet'].tolist()
labels = df['label'].tolist()

# Splitting dataset into train+val and test
train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=RAND_STATE)

# Splitting train+val into train and val
train_texts, validation_texts, train_labels, validation_labels = train_test_split(train_val_texts, train_val_labels, test_size=0.25, random_state=RAND_STATE)  # 0.25 x 0.8 = 0.2

# Creating datasets
sequence_length = 10  # Max number of words in a text
train_dataset = TextDataset(train_texts, train_labels, sequence_length)
validation_dataset = TextDataset(validation_texts, validation_labels, sequence_length)
test_dataset = TextDataset(test_texts, test_labels, sequence_length)

# Creating DataLoaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for data, label in train_loader:
    print(f"Train Encoded text: {data}")
    print(f"Train Label: {label}")
    break  # Just show one batch for brevity

for data, label in validation_loader:
    print(f"Validation Encoded text: {data}")
    print(f"Validation Label: {label}")
    break  # Just show one batch for brevity

for data, label in test_loader:
    print(f"Test Encoded text: {data}")
    print(f"Test Label: {label}")
    break  # Just show one batch for brevity

Train Encoded text: tensor([[ 9249,   288,  9182,  6467, 15831, 16784, 14985,  3744,  2095,  2886],
        [12493,  9746, 14026, 18670,  9588,  6219,  5640, 20155,  6219,  2317],
        [15738,   124,  6283, 20232, 11152,  8098,  4936,  7763, 16950,    25],
        [ 1674, 14415, 12894, 14680, 10873,  7407,  7368,  6192, 10002, 18437]])
Train Label: tensor([0, 1, 0, 1])
Validation Encoded text: tensor([[3925, 6913, 2028, 3912, 3591, 4467, 4922, 4487, 4383, 4484],
        [7583, 5973, 1990, 3053, 6689,  191, 4206, 7471, 1204, 3157],
        [8503, 6298, 2066, 3410, 1140, 3307, 6427,  718, 1589, 2886],
        [7583, 6020, 1990,  770, 3571, 7393,  792,  735, 8797, 6473]])
Validation Label: tensor([1, 1, 1, 1])
Test Encoded text: tensor([[2863, 7953, 4415,  651,  626, 2169,  223, 2173, 7409, 6529],
        [4909, 8420, 5283, 8077, 7102,   30, 1559, 5489, 2332, 3245],
        [6123,  223, 6925, 2041, 3443, 1095, 5059, 4281,  934, 1505],
        [7493, 7652, 6843,  453, 2045, 8095, 9025, 

In [6]:
# Parameters and Hyperparameters
vocabulary_size = 100000  # to adjust 
sequence_length = 50  # to adjust 
embedding_dim = 128
hidden_dim = 64
num_classes = 2
batch_size = 64
epochs = 2
learning_rate = 0.001

# Model, optimizer, and device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PolicyNetwork(vocabulary_size, embedding_dim, hidden_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    validate(model, device, validation_loader)

# After training, evaluate on the test set
test(model, device, test_loader)

Train Epoch: 1 [0/4752 (0%)]	Loss: 0.362245
Train Epoch: 1 [40/4752 (1%)]	Loss: 0.125249
Train Epoch: 1 [80/4752 (2%)]	Loss: 0.312006
Train Epoch: 1 [120/4752 (3%)]	Loss: 0.245866
Train Epoch: 1 [160/4752 (3%)]	Loss: 0.249490
Train Epoch: 1 [200/4752 (4%)]	Loss: 0.199117
Train Epoch: 1 [240/4752 (5%)]	Loss: 0.147369
Train Epoch: 1 [280/4752 (6%)]	Loss: 0.136874
Train Epoch: 1 [320/4752 (7%)]	Loss: 0.087879
Train Epoch: 1 [360/4752 (8%)]	Loss: 0.074132
Train Epoch: 1 [400/4752 (8%)]	Loss: 0.344574
Train Epoch: 1 [440/4752 (9%)]	Loss: 0.221542
Train Epoch: 1 [480/4752 (10%)]	Loss: 0.147119
Train Epoch: 1 [520/4752 (11%)]	Loss: 0.155488
Train Epoch: 1 [560/4752 (12%)]	Loss: 0.544138
Train Epoch: 1 [600/4752 (13%)]	Loss: 0.103073
Train Epoch: 1 [640/4752 (13%)]	Loss: 0.267107
Train Epoch: 1 [680/4752 (14%)]	Loss: 0.231547
Train Epoch: 1 [720/4752 (15%)]	Loss: 0.075307
Train Epoch: 1 [760/4752 (16%)]	Loss: 0.073378
Train Epoch: 1 [800/4752 (17%)]	Loss: 0.940467
Train Epoch: 1 [840/4752 (18%

In [7]:
# Parameters and Hyperparameters
n_trials=5
num_classes = 2

def objective(trial):
    # Define the search space
    # vocabulary_size = trial.suggest_categorical('vocabulary_size', [5000, 10000, 20000, 40000])
    vocabulary_size = 100000
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    embedding_dim = trial.suggest_categorical('embedding_dim', [64, 128, 256])
    hidden_dim = trial.suggest_categorical('hidden_dim', [32, 64, 128])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    step_size = trial.suggest_int('step_size', 1, 100)
    gamma = trial.suggest_float('gamma', 0.1, 1.0, log=True)
    sequence_length = trial.suggest_categorical('sequence_length', [50, 100, 200, 400])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Model setup with trial suggestions
    model = PolicyNetwork(vocabulary_size, embedding_dim, hidden_dim, num_classes, dropout_rate=dropout_rate).to(device)

    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=learning_rate)

    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    # Training loop
    epochs = 5  # Reduced for faster optimization cycles
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        val_loss, val_accuracy = validate(model, device, validation_loader)
        scheduler.step()

    # Set custom attributes for the trial
    trial.set_user_attr("val_loss", val_loss)
    trial.set_user_attr("val_accuracy", val_accuracy)
    
    # print(f"Returning from validate: val_loss={val_loss}, val_accuracy={val_accuracy}")
    # return val_loss

    # Objective: maximize validation accuracy by minimizing its negative value
    return -val_accuracy  # Return the negative accuracy

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials)  # Number of trials can be adjusted

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

# Retrieve the validation loss and accuracy from the best trial
best_val_loss = trial.user_attrs["val_loss"]
best_val_accuracy = trial.user_attrs["val_accuracy"]

print(f'Best Validation Loss: {best_val_loss}')
print(f'Best Validation Accuracy: {best_val_accuracy}')
print('Best Trial Parameters:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

[I 2024-04-02 21:31:12,844] A new study created in memory with name: no-name-6a524d41-e0ce-455f-a176-ab6a77a8383c


Train Epoch: 1 [0/4752 (0%)]	Loss: 0.347969
Train Epoch: 1 [320/4752 (7%)]	Loss: 0.403945
Train Epoch: 1 [640/4752 (13%)]	Loss: 0.445074
Train Epoch: 1 [960/4752 (20%)]	Loss: 0.310939
Train Epoch: 1 [1280/4752 (27%)]	Loss: 0.338621
Train Epoch: 1 [1600/4752 (34%)]	Loss: 0.400845
Train Epoch: 1 [1920/4752 (40%)]	Loss: 0.358307
Train Epoch: 1 [2240/4752 (47%)]	Loss: 0.306752
Train Epoch: 1 [2560/4752 (54%)]	Loss: 0.371696
Train Epoch: 1 [2880/4752 (60%)]	Loss: 0.372941
Train Epoch: 1 [3200/4752 (67%)]	Loss: 0.314638
Train Epoch: 1 [3520/4752 (74%)]	Loss: 0.348230
Train Epoch: 1 [3840/4752 (81%)]	Loss: 0.333738
Train Epoch: 1 [4160/4752 (87%)]	Loss: 0.291597
Train Epoch: 1 [4480/4752 (94%)]	Loss: 0.388255

Validation set: Average loss: 0.0220, Accuracy: 505/1584 (32%)

Train Epoch: 2 [0/4752 (0%)]	Loss: 0.373070
Train Epoch: 2 [320/4752 (7%)]	Loss: 0.356189
Train Epoch: 2 [640/4752 (13%)]	Loss: 0.411839
Train Epoch: 2 [960/4752 (20%)]	Loss: 0.370149
Train Epoch: 2 [1280/4752 (27%)]	Loss: 

[I 2024-04-02 21:31:34,300] Trial 0 finished with value: -0.32323232323232326 and parameters: {'batch_size': 32, 'learning_rate': 1.0118171703582376e-05, 'embedding_dim': 256, 'hidden_dim': 64, 'optimizer': 'SGD', 'dropout_rate': 0.18519460843451108, 'step_size': 71, 'gamma': 0.17167883086941504, 'sequence_length': 50}. Best is trial 0 with value: -0.32323232323232326.



Validation set: Average loss: 0.0220, Accuracy: 512/1584 (32%)

Train Epoch: 1 [0/4752 (0%)]	Loss: 0.339870
Train Epoch: 1 [640/4752 (13%)]	Loss: 0.377715
Train Epoch: 1 [1280/4752 (27%)]	Loss: 0.329393
Train Epoch: 1 [1920/4752 (40%)]	Loss: 0.340592
Train Epoch: 1 [2560/4752 (53%)]	Loss: 0.303404
Train Epoch: 1 [3200/4752 (67%)]	Loss: 0.375112
Train Epoch: 1 [3840/4752 (80%)]	Loss: 0.306475
Train Epoch: 1 [4480/4752 (93%)]	Loss: 0.306763

Validation set: Average loss: 0.0109, Accuracy: 1063/1584 (67%)

Train Epoch: 2 [0/4752 (0%)]	Loss: 0.368487
Train Epoch: 2 [640/4752 (13%)]	Loss: 0.302377
Train Epoch: 2 [1280/4752 (27%)]	Loss: 0.326948
Train Epoch: 2 [1920/4752 (40%)]	Loss: 0.363053
Train Epoch: 2 [2560/4752 (53%)]	Loss: 0.329449
Train Epoch: 2 [3200/4752 (67%)]	Loss: 0.325415
Train Epoch: 2 [3840/4752 (80%)]	Loss: 0.329252
Train Epoch: 2 [4480/4752 (93%)]	Loss: 0.288112

Validation set: Average loss: 0.0108, Accuracy: 1175/1584 (74%)

Train Epoch: 3 [0/4752 (0%)]	Loss: 0.331590
T